<a href="https://colab.research.google.com/github/Maximillian-Cat/Skin-Disease-Classification/blob/main/VGG%20Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import shutil
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/HAM/HAM_dataset
ham_df = pd.read_csv("HAM10000_metadata.csv") #read in the metadata
labels = ham_df.sort_values("dx") #sort the dataset by dx
class_names = list(labels.dx.unique()) #create a list of all unique dx in the dataset

/content/drive/MyDrive/HAM/HAM_dataset


In [ ]:
!mkdir /content/HAM #make a new HAM directory
%cd /content/HAM
for i in class_names:
    os.makedirs(os.path.join(i)) #create subfolders of all dx in the new HAM directory

/content/HAM


In [ ]:
for i in class_names: #loop through all dx
  for c in list(ham_df[ham_df["dx"] == i]["image_id"]): #loop through all Image ID in the metadata in accordance to its dx
    try:
      get_image_1 = os.path.join('/content/drive/MyDrive/HAM/HAM_dataset/HAM10000_images_part_1', c+'.jpg')
      move_image_1 = shutil.copy(get_image_1, '/content/HAM/'+i)
    except:
      get_image_2 = os.path.join('/content/drive/MyDrive/HAM/HAM_dataset/HAM10000_images_part_2', c+'.jpg')
      move_image_2 = shutil.copy(get_image_2, '/content/HAM/'+i) #same as above

#P/s: This code is quite inefficient and can take up to 1.5 hours to run, better solutions had yet to be found.
#This is understandable though considering it took ~2 hours to upload all pictures onto drive.


In [ ]:
!pip install split-folders
import splitfolders
input = "/content/HAM" #the starting position of the files
output = "/content/HAM" #the destination of the files

splitfolders.ratio(input, output = output, seed = 1337, ratio = (.8, .1, .1))
#splitting into train/val/test subfolders, more can be read here: https://pypi.org/project/split-folders/

Copying files: 0 files [00:00, ? files/s]


In [ ]:
!mkdir /content/drive/MyDrive/HAM_VGG/train
!mkdir /content/drive/MyDrive/HAM_VGG/test
!mkdir /content/drive/MyDrive/HAM_VGG/val

from google.colab import drive
import shutil

shutil.unpack_archive("/content/drive/MyDrive/HAM_VGG/train.tar.gz", "/content/drive/MyDrive/HAM_VGG/train")
shutil.unpack_archive("/content/drive/MyDrive/HAM_VGG/test.tar.gz", "/content/drive/MyDrive/HAM_VGG/test")
shutil.unpack_archive("/content/drive/MyDrive/HAM_VGG/val.tar.gz", "/content/drive/MyDrive/HAM_VGG/val")

mkdir: cannot create directory ‘/content/drive/MyDrive/HAM_VGG/train’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/HAM_VGG/test’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/HAM_VGG/val’: File exists


In [ ]:
import keras
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Conv2D, Dense, Flatten, MaxPool2D
from keras.models import Sequential, load_model
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.vgg16 import preprocess_input
import keras.metrics

In [ ]:
basemodel_19 = VGG19()
model_19 = tf.keras.Sequential(basemodel_19.layers[:-1])
model_19.add(tf.keras.layers.Dense(7, activation = 'softmax'))
for layer in basemodel_19.layers[:-1]:
    layer.trainable = False

traindata = ImageDataGenerator().flow_from_directory(directory = "/content/drive/MyDrive/HAM_VGG/train",
                                                     target_size = (224,224), class_mode = "categorical")
valdata = ImageDataGenerator().flow_from_directory(directory = "/content/drive/MyDrive/HAM_VGG/val",
                                                    target_size=(224,224), class_mode = "categorical")

optimizer = Adam(learning_rate = 0.001)
model_19.compile(optimizer = optimizer, loss = keras.losses.categorical_crossentropy, metrics = ['accuracy'])

checkpoint = ModelCheckpoint("vgg19_1.h5", monitor = 'val_accuracy', verbose = 1, save_best_only = True,
                             save_weights_only = False, mode = 'auto', period = 1)
earlystop = EarlyStopping(monitor = 'val_accuracy', min_delta = 0, patience = 20, verbose = 1, mode = 'auto')

hist_19 = model_19.fit_generator(steps_per_epoch = 100, generator = traindata, validation_data = valdata,
                                 validation_steps = 10, epochs = 50, callbacks = [checkpoint, earlystop])

574710816/574710816 [==============================] - 3s 0us/step
Found 8010 images belonging to 7 classes.
Found 998 images belonging to 7 classes.


<ipython-input-4-02a7d3f4d265>:21: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist_19 = model_19.fit_generator(steps_per_epoch = 100, generator = traindata, validation_data = valdata,


Epoch 1/50
100/100 [==============================] - ETA: 0s - loss: 1.0925 - accuracy: 0.6526
Epoch 1: val_accuracy improved from -inf to 0.65625, saving model to vgg19_1.h5
100/100 [==============================] - 84s 677ms/step - loss: 1.0925 - accuracy: 0.6526 - val_loss: 1.0059 - val_accuracy: 0.6562
Epoch 2/50
100/100 [==============================] - ETA: 0s - loss: 0.8633 - accuracy: 0.6986
Epoch 2: val_accuracy improved from 0.65625 to 0.70000, saving model to vgg19_1.h5
100/100 [==============================] - 64s 642ms/step - loss: 0.8633 - accuracy: 0.6986 - val_loss: 0.8803 - val_accuracy: 0.7000
Epoch 3/50
100/100 [==============================] - ETA: 0s - loss: 0.8001 - accuracy: 0.7250
Epoch 3: val_accuracy did not improve from 0.70000
100/100 [==============================] - 80s 797ms/step - loss: 0.8001 - accuracy: 0.7250 - val_loss: 1.0101 - val_accuracy: 0.6375
Epoch 4/50
100/100 [==============================] - ETA: 0s - loss: 0.7735 - accuracy: 0.7322


In [ ]:
!mv /content/vgg19_1.h5 /content/drive/MyDrive/HAM_VGG
#model_19 = load_model("/content/drive/MyDrive/HAM_VGG/vgg19_1.h5")
testdata = ImageDataGenerator().flow_from_directory(directory = "/content/drive/MyDrive/HAM_VGG/test",
                                                    target_size=(224,224), class_mode = "categorical")

from sklearn.metrics import classification_report, confusion_matrix
prediction = model_19.predict_generator(testdata)
prediction = np.argmax(prediction, axis = 1)

print('Confusion Matrix')
print(confusion_matrix(testdata.classes, prediction))

print('Classification Report')
target_names = ['Akiec', 'Bcc', 'Bkl','Df', 'Mel', 'Nv', 'Vasc',]
print(classification_report(testdata.classes, prediction, target_names = target_names))

Found 1007 images belonging to 7 classes.


<ipython-input-5-05d34a861fcf>:7: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  prediction = model_19.predict_generator(testdata)


Confusion Matrix
[[  3   1   1   0   4  24   1]
 [  1   4   7   0   9  31   0]
 [  2   5  13   1  13  76   1]
 [  0   0   1   1   1   9   0]
 [  2   0  16   0  19  72   3]
 [ 18  20  76   5  92 454   6]
 [  0   0   1   0   2  11   1]]
Classification Report
              precision    recall  f1-score   support

       Akiec       0.12      0.09      0.10        34
         Bcc       0.13      0.08      0.10        52
         Bkl       0.11      0.12      0.12       111
          Df       0.14      0.08      0.11        12
         Mel       0.14      0.17      0.15       112
          Nv       0.67      0.68      0.67       671
        Vasc       0.08      0.07      0.07        15

    accuracy                           0.49      1007
   macro avg       0.20      0.18      0.19      1007
weighted avg       0.49      0.49      0.49      1007



In [ ]:
basemodel_16 = VGG16()
model_16 = tf.keras.Sequential(basemodel_16.layers[:-1])
model_16.add(tf.keras.layers.Dense(7, activation = 'softmax'))
for layer in basemodel_16.layers[:-1]:
    layer.trainable = False

optimizer = Adam(learning_rate = 0.001)
model_16.compile(optimizer = optimizer, loss = keras.losses.categorical_crossentropy,
              metrics = ['accuracy'])

checkpoint = ModelCheckpoint("vgg16_1.h5", monitor = 'val_accuracy', verbose = 1, save_best_only = True,
                             save_weights_only = False, mode = 'auto', period = 1)
earlystop = EarlyStopping(monitor = 'val_accuracy', min_delta = 0, patience=20, verbose = 1, mode = 'auto')

hist_16 = model_16.fit_generator(steps_per_epoch = 100, generator = traindata, validation_data = valdata,
                                 validation_steps = 10, epochs = 50, callbacks = [checkpoint, earlystop])

553467096/553467096 [==============================] - 6s 0us/step


<ipython-input-6-39305b6b39b8>:15: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist_16 = model_16.fit_generator(steps_per_epoch = 100, generator = traindata, validation_data = valdata,


Epoch 1/50
100/100 [==============================] - ETA: 0s - loss: 1.0793 - accuracy: 0.6644
Epoch 1: val_accuracy improved from -inf to 0.69687, saving model to vgg16_1.h5
100/100 [==============================] - 65s 645ms/step - loss: 1.0793 - accuracy: 0.6644 - val_loss: 0.8356 - val_accuracy: 0.6969
Epoch 2/50
100/100 [==============================] - ETA: 0s - loss: 0.8245 - accuracy: 0.7122
Epoch 2: val_accuracy did not improve from 0.69687
100/100 [==============================] - 58s 577ms/step - loss: 0.8245 - accuracy: 0.7122 - val_loss: 0.9787 - val_accuracy: 0.6875
Epoch 3/50
100/100 [==============================] - ETA: 0s - loss: 0.8494 - accuracy: 0.7111
Epoch 3: val_accuracy improved from 0.69687 to 0.71250, saving model to vgg16_1.h5
100/100 [==============================] - 57s 569ms/step - loss: 0.8494 - accuracy: 0.7111 - val_loss: 1.1160 - val_accuracy: 0.7125
Epoch 4/50
100/100 [==============================] - ETA: 0s - loss: 0.7749 - accuracy: 0.7325


In [ ]:
!mv /content/vgg16_1.h5 /content/drive/MyDrive/HAM_VGG
#model_16 = load_model("/content/drive/MyDrive/HAM_VGG/vgg16_1.h5")

prediction = model_16.predict_generator(testdata)
prediction = np.argmax(prediction, axis=1)

print('Confusion Matrix')
print(confusion_matrix(testdata.classes, prediction))

print('Classification Report')
target_names = ['Akiec', 'Bcc', 'Bkl','Df', 'Mel', 'Nv', 'Vasc',]
print(classification_report(testdata.classes, prediction, target_names = target_names))

<ipython-input-7-74453f70e338>:4: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  prediction = model_16.predict_generator(testdata)


Confusion Matrix
[[  0   1   2   0   3  28   0]
 [  0   3   3   1   5  40   0]
 [  0   6   8   0   7  89   1]
 [  0   1   0   0   2   9   0]
 [  0   1   3   1   9  97   1]
 [  7  36  18   3  64 540   3]
 [  1   0   1   0   1  12   0]]
Classification Report
              precision    recall  f1-score   support

       Akiec       0.00      0.00      0.00        34
         Bcc       0.06      0.06      0.06        52
         Bkl       0.23      0.07      0.11       111
          Df       0.00      0.00      0.00        12
         Mel       0.10      0.08      0.09       112
          Nv       0.66      0.80      0.73       671
        Vasc       0.00      0.00      0.00        15

    accuracy                           0.56      1007
   macro avg       0.15      0.14      0.14      1007
weighted avg       0.48      0.56      0.51      1007



In [ ]:
import keras
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Conv2D, Dense, Flatten, MaxPool2D
from keras.models import Sequential, load_model
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.vgg16 import preprocess_input
import keras.metrics

In [ ]:
model = VGG16()
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
from os import listdir
import cv2
import numpy as np
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16
from keras.layers import Input, Flatten, Dense, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import random
from keras.preprocessing.image import ImageDataGenerator

raw_folder = "/content/drive/MyDrive/HAM_VGG/train"
def save_data(raw_folder=raw_folder):

    dest_size = (128, 128)
    print("Bắt đầu xử lý ảnh...")

    pixels = []
    labels = []

    # Lặp qua các folder con trong thư mục raw
    for folder in listdir(raw_folder):
        if folder!='.DS_Store':
            print("Folder=",folder)
            # Lặp qua các file trong từng thư mục chứa các em
            for file in listdir(raw_folder  + folder):
                if file!='.DS_Store':
                    print("File=", file)
                    pixels.append( cv2.resize(cv2.imread(raw_folder  + folder +"/" + file),dsize=(128,128)))
                    labels.append( folder)

    pixels = np.array(pixels)
    labels = np.array(labels)#.reshape(-1,1)

    from sklearn.preprocessing import LabelBinarizer
    encoder = LabelBinarizer()
    labels = encoder.fit_transform(labels)
    print(labels)

    file = open('pix.data', 'wb')
    # dump information to that file
    pickle.dump((pixels,labels), file)
    # close the file
    file.close()

    return

def load_data():
    file = open('pix.data', 'rb')

    # dump information to that file
    (pixels, labels) = pickle.load(file)

    # close the file
    file.close()

    print(pixels.shape)
    print(labels.shape)


    return pixels, labels

#save_data()
X,y = load_data()
#random.shuffle(X)
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=100)

print(X_train.shape)
print(y_train.shape)

def get_model():
    model_vgg16_conv = VGG16(weights='imagenet', include_top=False)

    # Dong bang cac layer
    for layer in model_vgg16_conv.layers:
        layer.trainable = False

    # Tao model
    input = Input(shape=(128, 128, 3), name='image_input')
    output_vgg16_conv = model_vgg16_conv(input)

    # Them cac layer FC va Dropout
    x = Flatten(name='flatten')(output_vgg16_conv)
    x = Dense(4096, activation='relu', name='fc1')(x)
    x = Dropout(0.5)(x)
    x = Dense(4096, activation='relu', name='fc2')(x)
    x = Dropout(0.5)(x)
    x = Dense(7, activation='softmax', name='predictions')(x)

    # Compile
    my_model = Model(inputs=input, outputs=x)
    my_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return my_model

vggmodel = get_model()

filepath="weights-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.1,
    rescale=1./255,
	width_shift_range=0.1,
    height_shift_range=0.1,
	horizontal_flip=True,
    brightness_range=[0.2,1.5], fill_mode="nearest")

aug_val = ImageDataGenerator(rescale=1./255)

vgghist=vggmodel.fit_generator(aug.flow(X_train, y_train, batch_size=64),
                               epochs=50,# steps_per_epoch=len(X_train)//64,
                               validation_data=aug.flow(X_test,y_test,
                               batch_size=64),
                               callbacks=callbacks_list)

vggmodel.save("vggmodel.h5")

FileNotFoundError: ignored